## ROS concepts explained Topic, Service, Parameter

Topic, Service, and Parameter are the main parts of ROS. Each simplifies some particular type of communication inside ROS so that your robots can do stuff efficiently.

In short:
**Topic** is used when we don't care who writes the message and who reads the message as long as it is on the right topic and of the right format (i.e. "What is the **temperature** outside?" - someone asks. "It is 5 **degrees Celcius**" answers someone) 

**Service** is a different format, where we ask for some calculation or action and receive response **in return**.
I.e. (I ask "how do you do, robot?", and I receive "fine, thank you").
Both command and response need to be in a format that we agreed upon. Also, there can be only one agent (i.e. one robot) with a particular service. 

**Parameter** is a common database of parameters that can be read or written by ROS nodes (programms). Such parameters may be the hardware configuration, robot description or some other settings. Usually, they are read by a node when initializing (starting).

In [ ]:
### we will again connect to the main server

%env ROS_MASTER_URI=http://igor-laptop-linux:11311
import rospy

### ROS Topic

Let's start from *ROS Topic*. You have used this protocol to read secret Ono message. **rostopic** program was smart enough to read this message automatically but normally we need to know what **message type** the topic has to read it. This is because the messages can be very different and actually have a whole structure inside. 
The message is **serialized** (coded) before being sent by **publisher** and then **deserialized** (decoded) by **subscriber**. The whole process is done automatically but we need to know the message type.

### Exercise: 
*Let's look again at the topics that are there. In the field below get the list of the topics with their message types in format [[topic_name,message_type],[... ]].*

In [1]:
topics = ... # read the topics here
print(topics)

*HINT:*
    You can find the function for finding published *topics* in the previous chapter: [setting_up_and_running_your_first_ROS_Node.ipynb](setting_up_and_running_your_first_ROS_Node.ipynb) 

You can see that there is topic called '/Ono/joint_state'. Let's read its  message stream using **rostopic** tool.

### Exercise:

*Read **Ono/joint_state** topic, using rostopic echo command, interupt kernel when fininshed*


In [3]:
! echo "put your command here to read /ono_joint_state topic,\n interupt kernel when you finished"

put your command here to read /ono_joint_state topic,
 interupt kernel when you finished


So this topic sends much more complicated messages. It uses one of common message types, called *Ono/joint_state*. As robots have multiple parts, connected using joints it is usually good to know how these joints are set. This can be used, for example, to visualize robot. The message has also a **header** that is a form of a **[timestamp](http://wiki.ros.org/ecl_time/Tutorials/Timestamps%20and%20Durations)** that gives us information when the message was sent. This is useful too, for example, use only the newest messages or even debug the communication if the header is strange or very old.

### Let's write our own publisher

Ok, let's publish some messages.

In [ ]:
import rospy
from std_msgs.msg import String

#rospy... # initialize your node here! otherwise ROS will not know who and where your node is!
rospy.init_node("my_name", anonymous=True)

Let's initialize the Publisher -- an object that publishes to the topic "chatter" a message in format of String. 

In [ ]:
my_publisher=rospy.Publisher("chatter",String,queue_size=10)

Ok, now let's publish some info. For simple messages like String, we can send the message super- easly.

In [ ]:
my_publisher.publish("My first message!")

Ok but how to read what we have sent? One way is to use *rostopic echo* command but you cannot do this from Ipython this time because it will only show you the messages sent after you have started the program. You can do this by opening a Ubuntu terminal, using the icon on the left ![Ubuntu terminal icon](images/terminal.png)
or shortcut Ctrl+Alt+T. Than put the command 

```rostopic echo /chatter``` 


When you re-run the publish command in Ipython, you should see your message appearing in the terminal.

Ok, with rospublish command our node became a ros publisher.
We can see that using rosnode.get_node_info. To see the whole graph structure right now we can use graphical tool called rqt_graph (you can also start it in a seperate terminal)

In [ ]:
!rqt_graph

What you see after running this command is whole ROS network graph. You can see your own node, among others, publishing on the /chatter topic. This type of communication allows for multiple senders and receivers.

### Let's write a Subscriber

**Subscriber** is an object that receives messages written on a particular topic. We want to receive all **calls** to our subscriber so we will register a **callback function** to do stuff when we receive a new message. The easiest thing we can do with an incoming message is to print it, so let's put a *print* function as our callback function.

After running the code below, you can check that your subscriber is working by using your previously defined publisher. Try to publish something to a /chatter topic!


In [ ]:
my_subscriber = rospy.Subscriber("chatter",String,print)

As soon as you create a new subscriber, you will start receiving a stream of messages and you will start to see the incoming messages. In case of "/chatter" topic they are published by your and other publishers, but imagine that there is an automatic publisher publishing every couple of messages. This could become overwhelming!

If you want to stop running the callback, use the unregister method.


In [ ]:
my_subscriber.unregister()

We can also write our own function and pass its name (its reference) to initialize Subscriber 

In [ ]:
# our own function, remember that String message is actually an object 
#with a .data field that actually consists the text string
def write_reversed(message):
    the_text=message.data #our String has a .data field
    print(the_text[::-1])
    

# now we create a new subscriber

new_subscriber= rospy.Subscriber("chatter",String,write_reversed)

This time, when you publish something to the "chatter" topic it will come reversed.

In [ ]:
new_subscriber.unregister()

Remember to unregister, otherwise multiple functions will run when you receive new message **even** if you write a new subscriber with the same name.

### what to do when you forget about that and create a new subscriber anyway?

No worries ;) we have a trick to cover you -- we will remove all the callbacks to the subscriber

In [ ]:
### all callbacks to a selected topics are stored in .impl.callbacks list, we can make this list empty
print(new_subscriber.impl.callbacks) # you can see what is there
new_subscriber.impl.callbacks=[]

### Exercise:
*Before we tackle this problem, try to write your own callback function to do something with received message. For example, read /how_long_is_my_message topic and print the length of the string*

*HINT 1:* You can define a function using the normal

    def some_function_name(message):
        the stuff you do

   and than pass the function _name_ as a callback so rospy.Subscriber(..., some_function_name)

*HINT 2:* Remember that the message is an object with a field data (message.data) that actually has the string itself


In [ ]:
def some_new_function(message):
    ... #here do your magic
    pass

subscriber2= ... # here create a subscriber that uses your function

In [ ]:
my_publisher.publish("My first letter!") #Now you can check

Now let's do something with the topic and publish the results back to the ROS

In [ ]:
publisher_reversed= rospy.Publisher('chatter_reversed',String,queue_size=10)

def reverse_and_publish(message):
    '''the function takes the incoming message, reverses it and publishes back to ROS'''
    
    reversed_text=message.data[::-1]
    publisher_reversed.publish(reversed_text)
    
    
subscriber= rospy.Subscriber("chatter",String,reverse_and_publish)    

subscriber_reversed = rospy.Subscriber("chatter_reversed",String,print)

In [ ]:
### now when we publish to the chatter topic, a new message will appear at chatter_reversed, we read
#it via subscriber_reversed

my_publisher.publish('scitobor evol I')

### Exercise:

Ok, nice! 

Now create a new publisher, to some choosen topic (you decide) and  use it to publish the length of received string. That is your new function will modify the incomming messages, something like

/how_long_is_my_message -> | your node | -> /message_length

*HINT:*
Call the newly created publisher from inside the callback function. 

*HINT 2* remember that if you publish as a String the field data needs to be an actual string not, for example integer. OR you can use other message types like int32



In [ ]:
subscriber_nowy.impl.callbacks=[]
subscriber_received.impl.callbacks=[]

In [ ]:
from std_msgs.msg import Int32 # use this type if you want to publish the length 


topic_name= ... #create a topic name 
publisher_len= rospy.Publisher(topic_name,Int32,queue_size=10) #here we create a publisher with Int32 type

def receive_and_modify(message):
    
    # here put your function logic 
    pass


subscriber_new = ... # here create your subscriber that uses your callbac function (receive_and_modify)



subscriber_received = rospy.Subscriber(topic_name,String,print)

In [ ]:
my_publisher.publish("this message has a length of 31")

In [ ]:
subscriber_new.unregister()

In [ ]:
subscriber_received.unregister()

Super nice! What you did is a common strategy used in ROS. Instead of one program that does all the computations, each node takes some messages from different topics and publishes your own. You connect them together and have a computation structure to, for example do robot localisation. 



### ROS  Service

Services are a common way to ask for some data/ calculation or change a state of the robot. 

Similar to ROS topic, both actors need to know the format of the message. There will be actually two formats, format of question and format of the answer. 

Let's see how such format looks

In [ ]:
!rossrv show ipython_robot_prototyping/Question 

What you can see is that for a question sent as a string 
the service will also respond with a string response.

You can talk with Ono using this format of the message. To do this, we will use a **proxy function**, that is we will define something that uses the service but from our perspective looks like a normal function that _returns_ some value. Try running this function couple of times. 

In [ ]:
import rospy
from ipython_robot_prototyping.srv import *


talk_with_ono = rospy.ServiceProxy('Ono/chat', Question)

Now we can talk with Ono using *talk_with_ono* function. Try it the function return value will be response from Ono, also Ono will say his response out loud ;)

In [ ]:
talk_with_ono("what do you do?") # modify this sentence 

As we can see, Ono is quite talkative. 



### ROS Param

The last (common) way of communication between nodes is [**ROS Param**](http://wiki.ros.org/Parameter%20Server). 

All nodes have access to a common _dictionary_ like structure, from which they can read and write parameters.

The typical scenario is that we write some setup parameters to ROS Parameter server instead than just to start a program with command line arguments. Other is to put elements that are used by multiple programs such as URDF robot models (nice example is here: [mymodelrobot.appspot.com](http://mymodelrobot.appspot.com)).

Think of ROS Param as a place to put settings and common data of your ROS network. 

Using ROS Parameter is done via rosparam module.

Let's see all current parameters

In [ ]:
import rosparam

rosparam.list_params("/")

We can also list subset of the parameters, by putting *namespace* after the /

In [ ]:
rosparam.list_params("/Ono") # this will only show Ono parameters

* rosparam.get_param * is for reading the parameters. We can read some particular parameters or sets of parameters

In [ ]:
print(rosparam.get_param("/Ono/age"))
print(rosparam.get_param("/Ono"))

### Additional info

Rosparam can keep different types of parameters like floats, strings, integers, list or dictionaries. 
Dictionaries are special as they will become *adressable*

While using .get_param, the parameter value must be a valid [YAML string](http://wiki.ros.org/rospy/Overview/Parameter%20Server) but you can use .get_param_raw to use python objects

In [ ]:
import math
## Some examples

# Setting a list parameter
rosparam.set_param("/Ono/list_of_nicknames","['the_bestest','knowledge_creator','robot_overlord']")

rosparam.set_param_raw("/Ono/favorite_number",math.pi)

rosparam.set_param_raw("/Ono/clothes_sizes",{"shoe":31,"hat":"Large","pants":12})

## we can address the elements of the dictionary directely by /Ono/clothes_sizes/garderobe_part
print(rosparam.get_param("Ono/clothes_sizes/pants"))


Normally your ROS Node would read such params at start, to set up things

In [ ]:
age= rosparam.get_param("/Ono/age")
if age>18:
    print("Ono can party hard 🍻")
else:
    print("Ono is too young to party 🚼")

You can change parameters by using
*rosparam.set_param(parameter_name, value)* This could influence all nodes that use this parameter.

You can delete parameters by using *rosparam.delete_param*


In [ ]:
rosparam.set_param("/Ono/brothers_age","100")
print(rosparam.get_param("/Ono/brothers_age"))
rosparam.delete_param("/Ono/brothers_age")

### Exercise:

 1. Add three different parameters about you. Your parameter address should start with your name or nickname. One of these parametes should be a dictionary. 
 2. Read these parameters
 3. Change one
 4. Delete one
 



## Wrapup